In [1]:
import torch

In [2]:
class term():
    joint_ids = ['FL_hip_joint', 'FR_hip_joint', 'RL_hip_joint', 'RR_hip_joint', 'FL_thigh_joint', 'FR_thigh_joint', 'RL_thigh_joint', 'RR_thigh_joint', 'FL_calf_joint', 'FR_calf_joint', 'RL_calf_joint', 'RR_calf_joint']#['RR_hip', 'RR_joint', 'RR_calf']
    num_envs = 64
    _num_legs = 4
    _prevision_horizon = 10

    def __init__(self):
        self._num_joints = len(self.joint_ids)
        self.f = torch.zeros(self.num_envs, self._num_legs)
        self.d = torch.zeros(self.num_envs, self._num_legs)
        self.p = torch.zeros(self.num_envs, self._num_legs, self._prevision_horizon)
        self.F = torch.zeros(self.num_envs, self._num_legs, self._prevision_horizon)
        self.z = [self.f, self.d, self.p, self.F]

        # create tensors for raw and processed actions
        self._raw_actions = torch.zeros(self.num_envs, self.action_dim2)
        self._processed_actions = torch.zeros_like(self.raw_actions)

    @property
    def action_dim(self) -> int:
        return self._num_joints
    
    @property
    def action_dim2(self) -> int:
        return self.f.shape[1:].numel() + self.d.shape[1:].numel() + self.p.shape[1:].numel() + self.F.shape[1:].numel()
    
    @property
    def action_dim3(self) -> int:
        return sum(variable.shape[1:].numel() for variable in self.z)
    
    @property
    def raw_actions(self) -> torch.Tensor:
        return self._raw_actions

    @property
    def processed_actions(self) -> torch.Tensor:
        return self._processed_actions

In [3]:
term1 = term()

In [4]:
term1.action_dim2

88

In [5]:
term1.action_dim3

88

In [6]:
term1.raw_actions.shape

torch.Size([64, 88])

In [7]:
term1.z[1]

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0.,

In [8]:
term1.num_envs

64

In [9]:
a = torch.zeros(term1.num_envs, term1.action_dim,2)
print('tensor :',a)
print('shape :', a.shape[1:].numel())

print(a.flatten().shape)

tensor : tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        ...,

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]])
shape : 24
torch.Size([1536])


In [10]:
from __future__ import annotations

from omni.isaac.orbit.assets import AssetBase
from omni.isaac.orbit.assets.articulation import Articulation
from omni.isaac.orbit_tasks.locomotion.model_based.model_based_env_cfg import LocomotionModelBasedEnvCfg
from omni.isaac.orbit_tasks.locomotion.model_based.config.unitree_aliengo.aliengo_base_env_cfg import UnitreeAliengoBaseEnvCfg

ModuleNotFoundError: No module named 'omni.physics'

In [ ]:
import jax.numpy as jnp
import jax
seed = 42
key = jax.random.key(seed)

In [ ]:
print(f'available devices: {torch.cuda.device_count()}')
print(f'current device: { torch.cuda.current_device()}')
torch.cuda.get_device_name(0)

available devices: 1
current device: 0


'NVIDIA GeForce RTX 3060'

In [ ]:
output_torques = (torch.rand(term1.num_envs, term1._num_joints, device='cuda') * 80) - 40
print('shape : ',output_torques.shape)
print('device : ',output_torques.device)

shape :  torch.Size([64, 12])
device :  cuda:0


In [ ]:
output_torques_jax = jax.random.normal(key=key, shape=output_torques.shape)

In [ ]:
print('--- Torch ---')
print('Shape : ', output_torques.shape)
print('Type : ', output_torques.type())
print('Type : ', type(output_torques))

print('')
print('--- Jax ---')
print('Shape : ', output_torques_jax.shape)
print('Type : ', type(output_torques_jax))

--- Torch ---
Shape :  torch.Size([64, 12])
Type :  torch.FloatTensor
Type :  <class 'torch.Tensor'>

--- Jax ---
Shape :  (64, 12)
Type :  <class 'jaxlib.xla_extension.ArrayImpl'>


In [ ]:
output_torques_jax.std()

Array(1.0063325, dtype=float32)

In [ ]:
import jax
import jax.dlpack
import torch
import torch.utils.dlpack

def jax_to_torch(x):
    return torch.utils.dlpack.from_dlpack(jax.dlpack.to_dlpack(x))
def torch_to_jax(x):
    return jax.dlpack.from_dlpack(torch.utils.dlpack.to_dlpack(x))

a = torch.tensor([1,2,3]).cuda()
a_jax = torch_to_jax(a)
print(a_jax)

[1 2 3]


In [ ]:
a.device

device(type='cuda', index=0)

In [ ]:
a_jax.devices()

{cuda(id=0)}

In [ ]:
output_torques = (torch.rand(term1.num_envs, term1._num_joints, device='cuda') * 80) - 40
print('shape : ',output_torques.shape)
print('device : ',output_torques.device)
print('Type : ', type(output_torques))

shape :  torch.Size([64, 12])
device :  cuda:0
Type :  <class 'torch.Tensor'>


In [ ]:
output_torques_jax = torch_to_jax(output_torques)
print('Shape : ', output_torques_jax.shape)
print('device : ',output_torques_jax.devices())
print('Type : ', type(output_torques_jax))

Shape :  (64, 12)
device :  {cuda(id=0)}
Type :  <class 'jaxlib.xla_extension.ArrayImpl'>


In [ ]:
def alo() -> tuple[int, int, str]:
    a = 2
    b = 3
    c = 4
    return a, b, str(c)

def alo2():
    a = 2
    b = 3
    c = 4
    return a, b, str(c)

In [ ]:
alo()

(2, 3, '4')

In [ ]:
alo2()

(2, 3, '4')

In [ ]:
print(type(alo()))
print(type(alo2()))

<class 'tuple'>
<class 'tuple'>


In [ ]:
a = tuple[2,3,4]
alo()

d, f, e = alo()
type(alo())

tuple

In [ ]:
import torch

a = torch.tensor([0, 1.21, 2])
b = torch.tensor([True, True, False])

shape = [2,3]
a = torch.rand(shape)
b = torch.empty(shape, dtype=torch.bool).bernoulli(0.5)


print(a.dtype)
print(b.dtype)
print(a)
print(b)

a*b

torch.float32
torch.bool
tensor([[0.5317, 0.4781, 0.3271],
        [0.3938, 0.3433, 0.9002]])
tensor([[ True, False,  True],
        [False,  True, False]])


tensor([[0.5317, 0.0000, 0.3271],
        [0.0000, 0.3433, 0.0000]])

In [ ]:
import torch

# Assuming you have a tensor of shape (batch_size, num_legs)
tensor = torch.randn(5, 4)  # Example tensor with shape (5, 4)

# Define the number of joints per leg
number_of_joint_per_leg = 3

# Modify the tensor to shape (batch_size, num_legs, number_of_joint_per_leg)
modified_tensor = torch.reshape(tensor, (tensor.shape[0], tensor.shape[1], number_of_joint_per_leg))

# Check the shape of the modified tensor
print("Modified tensor shape:", modified_tensor.shape)

RuntimeError: shape '[5, 4, 3]' is invalid for input of size 20

In [ ]:
import torch
import time

# Create some tensors for demonstration
T_shape = [4096,4,3]
c_shape = [4096,4]
T_1 = torch.rand(T_shape).cuda()
T_2 = torch.rand(T_shape).cuda()
c = torch.empty(c_shape, dtype=torch.bool).bernoulli(0.5).cuda()


# Create CUDA events
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

# Record start event
start_event.record()

# Example operation (e.g., matrix multiplication)

result = (T_1 * c.unsqueeze(-1)) + (T_2 * (~c).unsqueeze(-1))

# Record end event
end_event.record()

# Wait for computations to finish
torch.cuda.synchronize()

# Calculate elapsed time
elapsed_time = start_event.elapsed_time(end_event) / 1000  # Convert to seconds
print("Time taken:", elapsed_time, "seconds")

Time taken: 0.00016944000124931335 seconds


In [ ]:
import jax
def custom_operation(T_1, T_2, c_star):
    # Element-wise multiplication with c_star and its complement
    term1 = T_1 * c_star[..., None]
    term2 = T_2 * (~c_star)[..., None]
    
    # Sum the terms along the joint dimension
    T = term1 + term2
    
    return T

# Example usage
batch_size = 3
num_legs = 4
num_of_joints_per_leg = 5

# Random tensors for T_1, T_2, and c_star
T_1 = jax.random.normal(jax.random.PRNGKey(0), (batch_size, num_legs, num_of_joints_per_leg))
T_2 = jax.random.normal(jax.random.PRNGKey(1), (batch_size, num_legs, num_of_joints_per_leg))
c_star = jax.random.randint(jax.random.PRNGKey(2), (batch_size, num_legs), 0, 2)

# Perform custom operation
T = custom_operation(T_1, T_2, c_star)

print(T.shape)  # Output shape should be (batch_size, num_legs, num_of_joints_per_leg)

(3, 4, 5)


In [ ]:
import jax
import jax.numpy as jnp
from jax import jit

@jit
def custom_operation(T_1, T_2, c_star):
    # Element-wise multiplication with c_star and its complement
    term1 = T_1 * c_star[..., None]
    term2 = T_2 * (~c_star)[..., None]
    
    # Sum the terms along the joint dimension
    T = term1 + term2
    
    return T

# Example usage
batch_size = 3
num_legs = 4
num_of_joints_per_leg = 5

# Random tensors for T_1, T_2, and c_star
T_1 = jax.random.normal(jax.random.PRNGKey(0), (batch_size, num_legs, num_of_joints_per_leg))
T_2 = jax.random.normal(jax.random.PRNGKey(1), (batch_size, num_legs, num_of_joints_per_leg))
c_star = jax.random.randint(jax.random.PRNGKey(2), (batch_size, num_legs), 0, 2)

# Move tensors to GPU
T_1 = jax.device_put(T_1, jax.devices('gpu')[0])
T_2 = jax.device_put(T_2, jax.devices('gpu')[0])
c_star = jax.device_put(c_star, jax.devices('gpu')[0])

# Perform custom operation
T = custom_operation(T_1, T_2, c_star)

print(T.shape)  # Output shape should be (batch_size, num_legs, num_of_joints_per_leg)

(3, 4, 5)


In [ ]:
# Create some tensors for demonstration
batch_size = 4096
num_legs = 4
num_of_joints_per_leg = 3
T_1 = jax.random.normal(jax.random.PRNGKey(0), (batch_size, num_legs, num_of_joints_per_leg))
T_2 = jax.random.normal(jax.random.PRNGKey(1), (batch_size, num_legs, num_of_joints_per_leg))
c_star = jax.random.randint(jax.random.PRNGKey(2), (batch_size, num_legs), 0, 2)

# Create CUDA events
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

# Record start event
start_event.record()

# Example operation (e.g., matrix multiplication)
T = custom_operation(T_1, T_2, c_star)

# result = (T_1 * c.unsqueeze(-1)) + (T_2 * (~c).unsqueeze(-1))

# Record end event
end_event.record()

# Wait for computations to finish
torch.cuda.synchronize()

# Calculate elapsed time
elapsed_time = start_event.elapsed_time(end_event) / 1000  # Convert to seconds
print("Time taken:", elapsed_time, "seconds")

Time taken: 0.00010966400057077407 seconds


In [ ]:
a = [0,1,2,3,4,5,6,7,8,9,10]
num_legs = 4
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
print(a[:num_legs])
print(a[num_legs:2*num_legs])
print(a[2*num_legs:])

[0, 1, 2, 3]
[4, 5, 6, 7]
[8, 9, 10]


## Gait Generator
from : f, d, phase, time_horizon  
return : c, new_phase

In [ ]:
import torch

        Args:
            - f   (torch.Tensor): Leg frequency                         of shape(batch_size, num_legs, parallel_rollout)
            - d   (torch.Tensor): Stepping duty cycle                   of shape(batch_size, num_legs, parallel_rollout)
            - phase (tch.Tensor): phase of leg                          of shape(batch_size, num_legs, parallel_rollout)
            - time_horizon (int): Time horizon for the contact sequence

        Returns:
            - c     (torch.bool): Foot contact sequence                 of shape(batch_size, num_legs, time_horizon, parallel_rollout)
            - phase (tch.Tensor): The phase updated by one time steps   of shape(batch_size, num_legs, parallel_rollout)

In [ ]:
num_envs = 1
_num_legs = 2
parallel_rollout = 3
device = 'cpu'

time_horizon = 4
dt = 0.1

f = torch.zeros(num_envs, _num_legs, parallel_rollout, device=device)
f = torch.Tensor([1,2,3]).expand(num_envs,_num_legs, parallel_rollout)
print('---- Leg Frequency : f ----')
print('f shape:', f.shape)
print('f :', f)
print()

d = torch.zeros(num_envs, _num_legs, parallel_rollout, device=device)
d = d+2
print('---- Stepping duty cycle : d ----')
print('d shape:', d.shape)
print('d :', d)
print()

phase = torch.zeros(num_envs, _num_legs, parallel_rollout, device=device)
phase = torch.Tensor([1,2,3]).expand(num_envs,_num_legs, parallel_rollout)
print('---- Phase ----')
print('phase shape:', phase.shape)
print('phase :', phase)
print()

---- Leg Frequency : f ----
f shape: torch.Size([1, 2, 3])
f : tensor([[[1., 2., 3.],
         [1., 2., 3.]]])

---- Stepping duty cycle : d ----
d shape: torch.Size([1, 2, 3])
d : tensor([[[2., 2., 2.],
         [2., 2., 2.]]])

---- Phase ----
phase shape: torch.Size([1, 2, 3])
phase : tensor([[[1., 2., 3.],
         [1., 2., 3.]]])



In [ ]:
torch.linspace(start=1, end=time_horizon, steps=time_horizon)*dt

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [ ]:
print(phase.shape)
print(phase.unsqueeze(-1).shape)
print(phase.unsqueeze(-1).expand(*[-1] * len(phase.shape),time_horizon).shape)
phase.unsqueeze(-1).expand(num_envs,_num_legs,parallel_rollout,time_horizon)

torch.Size([1, 2, 3])
torch.Size([1, 2, 3, 1])
torch.Size([1, 2, 3, 4])


tensor([[[[1., 1., 1., 1.],
          [2., 2., 2., 2.],
          [3., 3., 3., 3.]],

         [[1., 1., 1., 1.],
          [2., 2., 2., 2.],
          [3., 3., 3., 3.]]]])

In [ ]:
new_phases = phase.unsqueeze(-1).expand(num_envs,_num_legs,parallel_rollout,time_horizon) + f.unsqueeze(-1).expand(num_envs,_num_legs,parallel_rollout,time_horizon)*torch.linspace(start=1, end=time_horizon, steps=time_horizon)*dt

print(new_phases.shape)
new_phases

torch.Size([1, 2, 3, 4])


tensor([[[[1.1000, 1.2000, 1.3000, 1.4000],
          [2.2000, 2.4000, 2.6000, 2.8000],
          [3.3000, 3.6000, 3.9000, 4.2000]],

         [[1.1000, 1.2000, 1.3000, 1.4000],
          [2.2000, 2.4000, 2.6000, 2.8000],
          [3.3000, 3.6000, 3.9000, 4.2000]]]])

In [ ]:
new_phases = new_phases%1

print(new_phases.shape)
new_phases

torch.Size([1, 2, 3, 4])


tensor([[[[0.1000, 0.2000, 0.3000, 0.4000],
          [0.2000, 0.4000, 0.6000, 0.8000],
          [0.3000, 0.6000, 0.9000, 0.2000]],

         [[0.1000, 0.2000, 0.3000, 0.4000],
          [0.2000, 0.4000, 0.6000, 0.8000],
          [0.3000, 0.6000, 0.9000, 0.2000]]]])

In [ ]:
new_phase = new_phases[...,0]

print(new_phase.shape)
new_phase

torch.Size([1, 2, 3])


tensor([[[0.1000, 0.2000, 0.3000],
         [0.1000, 0.2000, 0.3000]]])

In [ ]:
c = new_phases > d.unsqueeze(-1).expand(*[-1] * len(d.shape),time_horizon)

print(c.shape)
c.dim()

torch.Size([1, 2, 3, 4])


4

In [ ]:
def gait_generator(f, d, phase, time_horizon):
    new_phases = phase.unsqueeze(-1).expand(*[-1] * len(phase.shape),time_horizon) + f.unsqueeze(-1).expand(*[-1] * len(f.shape),time_horizon)*torch.linspace(start=1, end=time_horizon, steps=time_horizon)*dt

    new_phases = new_phases%1

    new_phase = new_phases[..., 0]

    c = new_phases > d.unsqueeze(-1).expand(*[-1] * len(d.shape),time_horizon)

    return c, new_phase


In [ ]:
import torch

# Create a tensor (size can be unknown)
tensor = torch.randn(1, 2, 1, 3, 1)

# Squeeze out singleton dimensions
unsqueezed_tensor = tensor.squeeze()

print("Original tensor shape:", tensor.shape)
print("Tensor shape after squeezing:", unsqueezed_tensor.shape)


Original tensor shape: torch.Size([1, 2, 1, 3, 1])
Tensor shape after squeezing: torch.Size([2, 3])


# Saved Hsitory of Development

In [ ]:
        # # parse the body index
        # body_ids, body_names = self._asset.find_bodies(self.cfg.body_name)
        # if len(body_ids) != 1:
        #     raise ValueError(
        #         f"Expected one match for the body name: {self.cfg.body_name}. Found {len(body_ids)}: {body_names}."
        #     )
        # # save only the first body index
        # self._body_idx = body_ids[0]
        # self._body_name = body_names[0]
        # # check if articulation is fixed-base
        # # if fixed-base then the jacobian for the base is not computed
        # # this means that number of bodies is one less than the articulation's number of bodies
        # if self._asset.is_fixed_base:
        #     self._jacobi_body_idx = self._body_idx - 1
        # else:
        #     self._jacobi_body_idx = self._body_idx
        # carb.log_info(  # log info for debugging
        #     f"Resolved body name for the action term {self.__class__.__name__}: {self._body_name} [{self._body_idx}]"
        # )

        # # convert the fixed offsets to torch tensors of batched shape
        # if self.cfg.body_offset is not None:
        #     self._offset_pos = torch.tensor(self.cfg.body_offset.pos, device=self.device).repeat(self.num_envs, 1)
        #     self._offset_rot = torch.tensor(self.cfg.body_offset.rot, device=self.device).repeat(self.num_envs, 1)
        # else:
        #     self._offset_pos, self._offset_rot = None, None

In [ ]:
   # Stolen from DifferentialInverseKinematicsAction
    def _compute_frame_jacobian(self):
        """Computes the geometric Jacobian of the target frame in the root frame.

        This function accounts for the target frame offset and applies the necessary transformations to obtain
        the right Jacobian from the parent body Jacobian.
        """
        # read the parent jacobian
        jacobian = self._asset.root_physx_view.get_jacobians()[:, self._jacobi_body_idx, :, self._joint_ids]

        jacobian = self._asset.root_physx_view.get_jacobians()

        """Ordered names of bodies in articulation (through rigid body view)."""
        prim_paths = self._asset.body_physx_view.prim_paths[: self._asset.num_bodies]
        body_names = [path.split("/")[-1] for path in prim_paths]
        print("Link names through body view: ", body_names) #['base', 'FL_hip', 'FL_thigh', 'FL_calf', 'FL_foot', 'FR_hip', 'FR_thigh', 'FR_calf', 'FR_foot', 'RL_hip', 'RL_thigh', 'RL_calf', 'RL_foot', 'RR_hip', 'RR_thigh', 'RR_calf', 'RR_foot']

        """Ordered names of bodies in articulation (through articulation view)."""
        body_names = self._asset.root_physx_view.shared_metatype.link_names
        print("Link names through articulation view: ", body_names) #['base', 'FL_hip', 'FR_hip', 'RL_hip', 'RR_hip', 'FL_thigh', 'FR_thigh', 'RL_thigh', 'RR_thigh', 'FL_calf', 'FR_calf', 'RL_calf', 'RR_calf', 'FL_foot', 'FR_foot', 'RL_foot', 'RR_foot']

        # account for the offset
        if self.cfg.body_offset is not None:
            # Modify the jacobian to account for the offset
            # -- translational part
            # v_link = v_ee + w_ee x r_link_ee = v_J_ee * q + w_J_ee * q x r_link_ee
            #        = (v_J_ee + w_J_ee x r_link_ee ) * q
            #        = (v_J_ee - r_link_ee_[x] @ w_J_ee) * q
            jacobian[:, 0:3, :] += torch.bmm(-math_utils.skew_symmetric_matrix(self._offset_pos), jacobian[:, 3:, :])
            # -- rotational part
            # w_link = R_link_ee @ w_ee
            jacobian[:, 3:, :] = torch.bmm(math_utils.matrix_from_quat(self._offset_rot), jacobian[:, 3:, :])

        return jacobian

In [ ]:
    def apply_actions(self):
        """Applies the actions to the asset managed by the term.
        Note: This is called at every simulation step by the manager.
        """
        output_torques = (torch.rand(self.num_envs, self._num_joints, device=self.device))# * 80) - 40

        # print('--- Torch ---')
        # print('shape : ',output_torques.shape)
        # print('device : ',output_torques.device)
        # print('Type : ', type(output_torques))
        
        output_torques_jax = torch_to_jax(output_torques)
        output_torques_jax = (output_torques_jax * 80) - 40

        # print('')
        # print('--- Jax ---')
        # print('Shape : ', output_torques_jax.shape)
        # print('device : ',output_torques_jax.devices())
        # print('Type : ', type(output_torques_jax))

        output_torques2 = jax_to_torch(output_torques_jax)

        # set joint effort targets (should be equivalent to torque) : Torque controlled robot
        self._asset.set_joint_effort_target(output_torques2, joint_ids=self._joint_ids)

In [ ]:
    def get_robot_state2(self):
        """ TODO Write description
        """

        # Joint Index
        fl_joints = self._asset.find_joints("FL.*")[0]		# list [0, 4,  8]
        fr_joints = self._asset.find_joints("FR.*")[0]		# list [1, 5,  9]
        rl_joints = self._asset.find_joints("RL.*")[0]		# list [2, 6, 10]
        rr_joints = self._asset.find_joints("RR.*")[0]		# list [3, 7, 11]

        # Body Index
        foot_idx = self._asset.find_bodies(".*foot")[0]

        # 'FL_foot', 'FR_foot', 'RL_foot', 'RR_foot'
        fl_jacobian = self._asset.root_physx_view.get_jacobians()[:, foot_idx[0], 0:3, fl_joints]# + 6]
        fr_jacobian = self._asset.root_physx_view.get_jacobians()[:, foot_idx[1], 0:3, fr_joints]# + 6]
        rl_jacobian = self._asset.root_physx_view.get_jacobians()[:, foot_idx[2], 0:3, rl_joints]# + 6]
        rr_jacobian = self._asset.root_physx_view.get_jacobians()[:, foot_idx[3], 0:3, rr_joints]# + 6]

        # foot position in wf
        fl_foot_pos_w = self._asset.data.body_state_w[:, foot_idx[0], 0:3]
        fr_foot_pos_w = self._asset.data.body_state_w[:, foot_idx[1], 0:3]
        rl_foot_pos_w = self._asset.data.body_state_w[:, foot_idx[2], 0:3]
        rr_foot_pos_w = self._asset.data.body_state_w[:, foot_idx[3], 0:3]

        # foot orientation in wf
        fl_foot_orient_w = self._asset.data.body_state_w[:, foot_idx[0], 3:7]
        fr_foot_orient_w = self._asset.data.body_state_w[:, foot_idx[1], 3:7]
        rl_foot_orient_w = self._asset.data.body_state_w[:, foot_idx[2], 3:7]
        rr_foot_orient_w = self._asset.data.body_state_w[:, foot_idx[3], 3:7]

        # Root state ``[pos, quat, lin_vel, ang_vel]`` in simulation world frame. Shape is (num_instances, 13)
        base_pose_w = self._asset.data.root_state_w[:, 0:3]
        base_orient_w = self._asset.data.root_state_w[:, 3:7]
        base_lin_vel_w = self._asset.data.root_state_w[:, 7:10]
        base_ang_vel_w = self._asset.data.root_state_w[:, 10:13]

        # foot position, orientation in bf
        fl_foot_pos_b, fl_foot_orient_b = math_utils.subtract_frame_transforms(base_pose_w, base_orient_w, fl_foot_pos_w, fl_foot_orient_w)
        fr_foot_pos_b, fr_foot_orient_b = math_utils.subtract_frame_transforms(base_pose_w, base_orient_w, fr_foot_pos_w, fr_foot_orient_w)
        rl_foot_pos_b, rl_foot_orient_b = math_utils.subtract_frame_transforms(base_pose_w, base_orient_w, rl_foot_pos_w, rl_foot_orient_w)
        rr_foot_pos_b, rr_foot_orient_b = math_utils.subtract_frame_transforms(base_pose_w, base_orient_w, rr_foot_pos_w, rr_foot_orient_w)

        # foot joint position
        fl_joint_pos = self._asset.data.joint_pos[:, fl_joints]
        fr_joint_pos = self._asset.data.joint_pos[:, fr_joints]
        rl_joint_pos = self._asset.data.joint_pos[:, rl_joints]
        rr_joint_pos = self._asset.data.joint_pos[:, rr_joints]

        # foot joint velocity
        fl_joint_vel = self._asset.data.joint_vel[:, fl_joints]
        fr_joint_vel = self._asset.data.joint_vel[:, fr_joints]
        rl_joint_vel = self._asset.data.joint_vel[:, rl_joints]
        rr_joint_vel = self._asset.data.joint_vel[:, rr_joints]

        print('alo')

# Stance Leg controller

In [ ]:
import torch

# Define symbolic variables
batch_size = 2
num_legs = 4
num_joints_per_leg = 5

# Instantiate the tensor with symbolic shape
shape = (batch_size, num_legs, 3, num_joints_per_leg)
tensor = torch.randn(*shape)

print("Random Tensor Shape:", tensor.shape)
print("Random Tensor:")
print(tensor)


Random Tensor Shape: torch.Size([2, 4, 3, 5])
Random Tensor:
tensor([[[[-1.0008,  0.7120, -0.1742, -1.4273, -0.2322],
          [ 0.4963, -0.0760,  0.7140,  0.5691, -0.4790],
          [-0.2406,  0.7622,  0.0441, -2.3687,  0.6843]],

         [[-1.1865, -0.2030, -0.8373,  1.5632, -0.9582],
          [-1.6118,  0.9994,  0.2950,  1.7622, -0.6367],
          [-0.9983,  0.1925,  1.1585, -0.3202, -0.7383]],

         [[ 0.9787,  0.0971,  0.9300, -1.4056,  0.0978],
          [-0.8115,  0.1189,  1.5842,  0.1966,  1.0970],
          [-0.2550, -2.1972, -0.2008, -0.6315,  0.5687]],

         [[ 0.8719, -0.0730, -0.2174,  0.8004, -0.7905],
          [ 0.4587, -0.1622,  0.8989,  1.1916,  0.8667],
          [ 0.1111, -1.7449,  0.0858,  1.7471,  0.4848]]],


        [[[-2.6416,  1.3883,  1.4118, -0.1556, -0.0260],
          [ 0.0520, -1.5354, -0.1618, -0.0765,  0.7076],
          [ 0.6039, -0.3753, -0.7691,  0.4189, -1.3773]],

         [[-0.7048,  1.1861, -0.2367, -0.0804,  0.8997],
          [ 1.2

In [ ]:
c0 = torch.randint(0, 2, (batch_size, num_legs), dtype=torch.bool)

print("Boolean Tensor Shape:", c0.shape)
print("Boolean Tensor:")
print(c0)

Boolean Tensor Shape: torch.Size([2, 4])
Boolean Tensor:
tensor([[ True,  True, False, False],
        [False,  True, False,  True]])


In [ ]:
jacobian = torch.randn(batch_size, num_legs, 3, num_joints_per_leg)
F0_star = torch.randn(batch_size, num_legs, 3)

In [ ]:
# jacobian = torch.randn(batch_size, num_legs, 3, num_joints_per_leg)
jacobian_T = jacobian.transpose(-1,-2).detach().clone() # Transpose last two dimensions
print('Jacobian   shape :',jacobian.shape)
print('Jacobian.T shape :',jacobian_T.shape)

# F0_star = torch.randn(batch_size, num_legs, 3)
F0_star2 = F0_star.unsqueeze(-1).detach().clone() 
print('   GRF     shape :',F0_star.shape)
print('   GRF 2   shape :',F0_star2.shape)

q = torch.matmul(jacobian_T, F0_star.unsqueeze(-1))
q2 = q.squeeze(-1).clone().detach()
print('     q     shape :',q.shape)
print('     q2    shape :',q2.shape)

print('')
print('Jacobian : ')
print(jacobian[1,3,:,:])

print('')
print('GRF : ')
print(F0_star[1,3,:])

print('')
print('Joints : ')
print(q2[1,2,:])

T = q2 * ~c0.unsqueeze(-1).expand(*[-1] * len(c0.shape), T.shape[-1])
print('')
print('Torques : ')
print(T[:,:,:])

T.shape[-1]

Jacobian   shape : torch.Size([2, 4, 3, 5])
Jacobian.T shape : torch.Size([2, 4, 5, 3])
   GRF     shape : torch.Size([2, 4, 3])
   GRF 2   shape : torch.Size([2, 4, 3, 1])
     q     shape : torch.Size([2, 4, 5, 1])
     q2    shape : torch.Size([2, 4, 5])

Jacobian : 
tensor([[ 1.8449, -0.1331, -0.4473, -0.3967, -0.5775],
        [-0.1599, -0.0411,  0.1473, -0.2492,  1.2638],
        [-1.4937, -1.0205, -0.9950,  0.1527,  1.4755]])

GRF : 
tensor([ 0.8727, -0.0659, -0.1853])

Joints : 
tensor([-1.1362, -1.4506, -0.6089,  2.4870,  1.3955])

Torques : 
tensor([[[-0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
         [ 0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
         [-1.8240,  0.8967, -0.6014,  0.1919,  0.6543],
         [-0.0195, -0.1334, -1.3107,  0.5556, -2.4532]],

        [[ 0.4518,  1.8152,  1.0578, -2.1407,  0.1980],
         [-0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
         [-1.1362, -1.4506, -0.6089,  2.4870,  1.3955],
         [ 0.0000,  0.0000, -0.0000, -0.0000, -0

5

In [ ]:
print(T.shape)
print(T)
print(T[:,:,4])

torch.Size([2, 4, 5])
tensor([[[-0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
         [ 0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
         [-1.8240,  0.8967, -0.6014,  0.1919,  0.6543],
         [-0.0195, -0.1334, -1.3107,  0.5556, -2.4532]],

        [[ 0.4518,  1.8152,  1.0578, -2.1407,  0.1980],
         [-0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
         [-1.1362, -1.4506, -0.6089,  2.4870,  1.3955],
         [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]])
tensor([[-0.0000,  0.0000,  0.6543, -2.4532],
        [ 0.1980, -0.0000,  1.3955, -0.0000]])


In [ ]:
import torch

# Define symbolic variables
batch_size = 2
num_legs = 4
num_joints_per_leg = 5

jacobian = torch.randn(batch_size, num_legs, 3, num_joints_per_leg)
jacobian_dot = torch.randn(batch_size, num_legs, 3, num_joints_per_leg)
q_dot = torch.randn(batch_size, num_legs, num_joints_per_leg)
mass_matrix = torch.randn(batch_size, num_legs, num_joints_per_leg, num_joints_per_leg)
h = torch.randn(batch_size, num_legs, num_joints_per_leg)

print("Jacobian     Shape:", jacobian.shape)
print("Jacobian dot Shape:", jacobian_dot.shape)
print("     q   dot Shape:", q_dot.shape)
print("Mass Matrix  Shape:", mass_matrix.shape)
print("     h       Shape:", h.shape)

Jacobian     Shape: torch.Size([2, 4, 3, 5])
Jacobian dot Shape: torch.Size([2, 4, 3, 5])
     q   dot Shape: torch.Size([2, 4, 5])
Mass Matrix  Shape: torch.Size([2, 4, 5, 5])
     h       Shape: torch.Size([2, 4, 5])


In [ ]:
J_dot_x_q_dot = torch.matmul(jacobian_dot, q_dot.unsqueeze(-1)).squeeze(-1)
print("                   J_dot_x_q_dot Shape:", J_dot_x_q_dot.shape)

jacobian_inv = torch.linalg.pinv(jacobian)
print("                   jacobian_inv  Shape:", jacobian_inv.shape)

J_inv_p_dot_dot_min_J_dot_x_q_dot = torch.matmul(jacobian_inv, J_dot_x_q_dot.unsqueeze(-1)).squeeze(-1)
print("     J⁻¹[p_dot_dot - J(q)*q_dot] Shape:", J_inv_p_dot_dot_min_J_dot_x_q_dot.shape)

M_J_inv_p_dot_dot_min_J_dot_x_q_dot = torch.matmul(mass_matrix, J_inv_p_dot_dot_min_J_dot_x_q_dot.unsqueeze(-1)).squeeze(-1)
print('M(q)*J⁻¹[p_dot_dot - J(q)*q_dot] Shape:', M_J_inv_p_dot_dot_min_J_dot_x_q_dot.shape)

# Final step
T = torch.add(M_J_inv_p_dot_dot_min_J_dot_x_q_dot, h)
print('                              T  Shape:', T.shape)

                   J_dot_x_q_dot Shape: torch.Size([2, 4, 3])
                   jacobian_inv  Shape: torch.Size([2, 4, 5, 3])
     J⁻¹[p_dot_dot - J(q)*q_dot] Shape: torch.Size([2, 4, 5])
M(q)*J⁻¹[p_dot_dot - J(q)*q_dot] Shape: torch.Size([2, 4, 5])
                              T  Shape: torch.Size([2, 4, 5])


## Jacobian dim

In [ ]:
import torch

# Define symbolic variables
batch_size = 5
num_legs = 4
num_joints_per_leg = 3

_foot_idx = [13,14,15,16]
_joint_idx = [[0,4,8],[1,5,9],[2,6,10],[3,7,11]]
# _joint_idx = [[[[0,4,8]],[[1,5,9]],[[2,6,10]],[[3,7,11]]]]
_joint_idx_tensor = torch.Tensor(_joint_idx)

jacobian = torch.randn(batch_size, 17, 6, 18)
print(jacobian.shape)

torch.Size([5, 17, 6, 18])


In [ ]:
print(jacobian[:,_foot_idx,:3,:].shape)

for leg_i, joints_in_leg_i in enumerate(_joint_idx):
    print(leg_i)
    print(joints_in_leg_i)

torch.Size([5, 4, 3, 18])
0
[0, 4, 8]
1
[1, 5, 9]
2
[2, 6, 10]
3
[3, 7, 11]


In [ ]:
import numpy as np
jacob1 = jacobian[:, 0, :3, [0,4,8]].unsqueeze(1)
jacob2 = jacobian[:, 1, :3, [1,5,9]].unsqueeze(1)
jacob3 = jacobian[:, 2, :3, [2,6,10]].unsqueeze(1)
print(jacob1.shape)
print(torch.cat((jacob1, jacob2, jacob3), dim=1).shape)

jacob = []
jacob.append(jacobian[:, 0, :3, 6+np.asarray([0,4,8])].unsqueeze(1))
jacob.append(jacobian[:, 1, :3, [1,5,9]].unsqueeze(1))
jacob.append(jacobian[:, 2, :3, [2,6,10]].unsqueeze(1))
print(torch.cat((jacob), dim=1).shape)

torch.Size([5, 1, 3, 3])
torch.Size([5, 3, 3, 3])
torch.Size([5, 3, 3, 3])


In [ ]:
12//3

4

In [ ]:
import torch

a = torch.tensor([[[1.0, 2.0, 2.1], [3.0, 4.0, 4.1]], [[5,6, 6.1] , [7, 8, 8.1]]])
print(a)
print()
a.permute(0,2,1).reshape(2,6)
# a.view(2,6)

tensor([[[1.0000, 2.0000, 2.1000],
         [3.0000, 4.0000, 4.1000]],

        [[5.0000, 6.0000, 6.1000],
         [7.0000, 8.0000, 8.1000]]])



tensor([[1.0000, 3.0000, 2.0000, 4.0000, 2.1000, 4.1000],
        [5.0000, 7.0000, 6.0000, 8.0000, 6.1000, 8.1000]])

# Extract Touch Down Pos : Convolutions

        # Utiliser une convolution sur c (contact sequence) pour trouver le point de départ et d'arriver du pied.
        # Avec un filtre genre f = [0, 1], pour ne garder que les flancs montants
        # Imaginons p = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]
        #           c = [ 1,  1,  0,  0,  0,  1,  1,  0,  0,   1]
        # Les points de départ serait p1, p6 et p10 les points d'arrivé p6 et p10
        # Il faudrait retourner qqch comme 
        #        key =  [1,   0,  0,  0,  0,  1,  0,  0,  0,   1]
        # Qui permetrait d'extraire facilement [p1, p6, p10] avec p[key]

In [ ]:
import torch

batch_size = 2
num_legs = 4
time_horizon = 10

c = torch.empty(batch_size, num_legs, time_horizon).bernoulli(0.2)
c

tensor([[[0., 0., 0., 0., 1., 0., 0., 0., 1., 1.],
         [0., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
         [0., 0., 0., 1., 0., 1., 0., 0., 1., 0.]]])

In [ ]:
p = torch.randn(c.shape)
p2 = p * c
p2

tensor([[[ 0.0000, -0.0000, -0.0000, -0.0000,  0.2320, -0.0000, -0.0000,
           0.0000,  0.2659, -1.4655],
         [ 0.0000, -0.0000,  0.6376,  0.0000,  0.0000, -0.0000,  0.0000,
           0.0000,  1.6942,  0.0000],
         [ 0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000,
           0.0000,  1.2197,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          -0.0000, -0.3521,  0.0000],
         [ 0.6727,  0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.4122,
           0.0000, -0.0000,  0.0000],
         [ 1.9076,  0.0000,  0.0000, -0.0000,  3.6689, -0.0000, -0.4448,
          -0.0000,  0.4419,  0.0000],
         [-0.0000,  0.0000, -0.0000, -0.7543, -0.0000,  0.6289,  0.0000,
           0.0000, -1.0148,  0.0000]]])

In [ ]:
indx = torch.argmax((c!=0).float(), dim=-1)
indx

tensor([[4, 2, 0, 8],
        [8, 0, 0, 3]])

In [ ]:
torch.gather(p, -1, indx.unsqueeze(-1)).squeeze(-1)

tensor([[ 0.2320,  0.6376,  1.4360,  1.2197],
        [-0.3521,  0.6727,  1.9076, -0.7543]])

In [ ]:
kernel = torch.tensor([-1, 1], dtype=torch.float32)
kernel = kernel.unsqueeze(0).unsqueeze(0).expand(4,1,2)
torch.conv1d(c, kernel, groups=4, padding=(0,1))

RuntimeError: expected padding to be a single integer value or a list of 1 values to match the convolution dimensions, but got padding=[0, 1]

In [ ]:
print(kernel)
print(kernel.shape)
print()

kernel2 = kernel.unsqueeze(0).unsqueeze(0).expand(4,1,2)
print(kernel2)
print(kernel2.shape)
print()

tensor([-1.,  1.])
torch.Size([2])

tensor([[[-1.,  1.]],

        [[-1.,  1.]],

        [[-1.,  1.]],

        [[-1.,  1.]]])
torch.Size([4, 1, 2])



In [ ]:
import torch
a = torch.tensor([[[1,1,1,1],[2,2,2,2],[3,3,3,3]], [[4,4,4,4], [5,5,5,5], [6,6,6,6]]]).transpose(-1,-2)
b = torch.tensor([[[1,2,3,4],[2,2,2,2],[3,3,3,3]], [[4,4,4,4], [5,5,5,5], [6,6,6,6]]]).transpose(-1,-2)
print(a.shape)

c = torch.cat((a,b), dim=1)
c.shape




torch.Size([2, 4, 3])


torch.Size([2, 8, 3])

In [ ]:
import torch
f = torch.tensor([[1,2,3], [4, 5, 6]])
d = torch.tensor([[0.1,0.2,0.3], [0.4, 0.5, 0.6]])
print(f.shape)
print(d.shape)

swing_period = ((1-d) / f) + 0.07
print(swing_period.shape)
print(swing_period)
print()
print(1-d)
print()
print((1-d)/f)
print()
print((1-d)/f + 0.07)

torch.Size([2, 3])
torch.Size([2, 3])
torch.Size([2, 3])
tensor([[0.9700, 0.4700, 0.3033],
        [0.2200, 0.1700, 0.1367]])

tensor([[0.9000, 0.8000, 0.7000],
        [0.6000, 0.5000, 0.4000]])

tensor([[0.9000, 0.4000, 0.2333],
        [0.1500, 0.1000, 0.0667]])

tensor([[0.9700, 0.4700, 0.3033],
        [0.2200, 0.1700, 0.1367]])


In [ ]:
f[:,-1] = 1
f

tensor([[1, 2, 1],
        [4, 5, 1]])

In [ ]:
c = torch.empty(2, 3).bernoulli(0.5).bool()
print(c)

tensor([[ True, False, False],
        [False, False, False]])


In [ ]:
(f * c) + (d * ~c)

tensor([[1.0000, 0.2000, 0.3000],
        [0.4000, 0.5000, 0.6000]])

In [ ]:
print(a.shape[:-1])
x = (a[:,:,:2] + b[:,:,:2]) / 2
torch.cat((x,0.05*torch.ones_like(a[:,:, :1])), dim=2)

torch.Size([2, 4])


tensor([[[1.0000, 2.0000, 0.0500],
         [1.5000, 2.0000, 0.0500],
         [2.0000, 2.0000, 0.0500],
         [2.5000, 2.0000, 0.0500]],

        [[4.0000, 5.0000, 0.0500],
         [4.0000, 5.0000, 0.0500],
         [4.0000, 5.0000, 0.0500],
         [4.0000, 5.0000, 0.0500]]])

In [ ]:
print(d)
print()
print(f)
print()

d**f

tensor([[0.1000, 0.2000, 0.3000],
        [0.4000, 0.5000, 0.6000]])

tensor([[1, 2, 3],
        [4, 5, 6]])



tensor([[0.1000, 0.0400, 0.0270],
        [0.0256, 0.0312, 0.0467]])

In [ ]:
print(a.shape)
print()
print(a)

print(a[..., 1])
print(a[..., 1].shape)
print()

print(a[:,:, 1])
print(a[:,:, 1].shape)
print()

torch.Size([2, 4, 3])

tensor([[[1, 2, 3],
         [1, 2, 3],
         [1, 2, 3],
         [1, 2, 3]],

        [[4, 5, 6],
         [4, 5, 6],
         [4, 5, 6],
         [4, 5, 6]]])
tensor([[2, 2, 2, 2],
        [5, 5, 5, 5]])
torch.Size([2, 4])

tensor([[2, 2, 2, 2],
        [5, 5, 5, 5]])
torch.Size([2, 4])



In [ ]:
import jax
jax.devices()

import os
print(os.environ.get("CUDA_HOME"))  # Check CUDA_HOME
print(os.environ.get("LD_LIBRARY_PATH"))  # Check LD_LIBRARY_PATH (Linux/macOS)
print(os.environ.get("PATH"))  # Check PATH (Windows)

print('-------------')
import torch
print(torch.backends.cudnn.version())

In [ ]:
import torch

""" Given feet position sequence and contact sequence -> compute swing trajectories by fitting a cubic spline between
the lift-off and the touch down define in the contact sequence. 
- Swing frequency and duty cycle are used to compute the swing period
- A middle point is used for the interpolation : which is heuristically defined. It defines the step height
- p1 (middle point) and p2 (touch-down) are updated each time, while p0 is conserved (always the same lift off position)

Args:
    - p   (torch.Tensor): Foot position sequence                of shape(batch_size, num_legs, 3, time_horizon)
    - c   (torch.Tensor): Foot contact sequence                 of shape(c)
    - f   (torch.Tensor): Leg frequency                         of shape(batch_size, num_legs)
    - d   (torch.Tensor): Stepping duty cycle                   of shape(batch_size, num_legs)
    - decimation   (int): Number of timestep for the traj.

Returns:
    - pt  (torch.Tensor): Desired Swing Leg trajectories        of shape(batch_size, num_legs, 9, decimation)   (9 = xyz_pos, xzy_vel, xyz_acc)
"""

batch_size = 3
num_legs = 4
time_horizon = 5
dt_out = 0.1

c = torch.randn(batch_size, num_legs, time_horizon)
c_prev = c
p0 = torch.randn(batch_size, num_legs, 3)
swing_time = torch.randn(batch_size, num_legs)


# Step 0. Define and Compute usefull variables

# Heuristic TODO Save that on the right place, could also be a RL variable
step_height = 0.05

# Time during wich the leg is in swing. TODO Why +0.07 ? Is it an heuristic also ?
# Shape (batch_size, num_legs)
swing_period = ((1-d) / f) + 0.07

half_swing_period = swing_period / 2
time_fac = 1 / (swing_period / 2) #bezier_time_factor


# Step 1. Retrieve the three interpolation points : p0, p1, p2 (lift-off, middle point, touch down)

# Retrieve p0 : If c(0)=0 and c(-1)=1 : The leg lift-off -> p0 = p(0) # TODO p(0) or must it be from simulation data ? TODO Must it be p(0) or p(-1)
# Update only the p0 that are new lift off positions
# shape (batch_size, num_legs, 3) 
lifting_off = (c[:,:,0]==0) * (c_prev == 1)
p0 = (p[:,:,:,0] * lifting_off) + (p0 * ~lifting_off)  

# Retrieve p2 : Retrieve the index of the touch down in the contact sequence : First Non-zero Index
# Set the last value of c as ONE to avoid the case of only 0 in the contact sequence, wich return the first element (make more sense to retrun the last)
# With the touch_down index, retrieve the touch down foot position : p2
# shape (batch_size, num_legs, 3) 
c[:,:,-1] = 1 # TODO Does it modify c also outside this function ?
first_non_zero_indx = torch.argmax((c!=0).float(), dim=-1)
p2 = torch.gather(p, -1, first_non_zero_indx.unsqueeze(-1)).squeeze(-1)

# Retrieve p1 : (x,y) position are define as the middle point between p0 and p1 (lift-off and touch-down). z is heuristcally define
# shape (batch_size, num_legs, 3)
p1 = (p0[:,:,:2] + p2[:,:,:2]) / 2     # p1(x,y) is in the middle of p0 and p2
p1 = torch.cat((p1, step_height*torch.ones_like(p1[:,:,:1])), dim=2) # Append a third dimension z : defined as step_height

# Step 2. Compute the parameters for the interpolation

# Swing time : reset if lifting off, then increment by one time step (outer loop)
# then compute t in [0, Delta_t/2], which would be use for the spline interpolation
# shape (batch_size, num_legs)
swing_time = (swing_time * ~lifting_off) + dt_out
t = swing_time % half_swing_period  # Swing time (half)

# Compute the a,b,c,d polynimial coefficient for the cubic interpolation S(t) = a*t^3 + b*t^2 + c*t + d
# If swing_time < swing period/2 -> S_0(t) (ie. first interpolation), otherwise -> S_1(t - delta_t/2) (ie. second interpolation)
# cp_x shape (batch_size, num_legs, 3)
is_S0 = (swing_time <=  half_swing_period).unsqueeze(-1).expand(*[-1] * len(swing_time.shape), 3)  # shape (batch_size, num_legs, 3)
cp1 = (p0 * is_S0)                                         + (p1 * ~is_S0)
cp2 = (p0 * is_S0)                                         + (torch.cat((p2[:,:,:2], p1[:,:,2:]), dim=2)* ~is_S0)
cp3 = (torch.cat((p0[:,:,:2], p1[:,:,2:]), dim=2) * is_S0) + (p2 * ~is_S0)
cp4 = (p1 * is_S0)                                              + (p2 * ~is_S0)

# Step 3. Compute the interpolation trajectory
desired_foot_pos_traj = cp1*(1 - time_fac*t)**3 + 3*cp2*(time_fac*t)*(1 - time_fac*t)**2 + 3*cp3*((time_fac*t)**2)*(1 - time_fac*t) + cp4*(time_fac*t)**3
desired_foot_vel_traj = 3*(cp2 - cp1)*(1 - time_fac*t)**2 + 6*(cp3 - cp2)*(1 - time_fac*t)*(time_fac*t) + 3*(cp4 - cp3)*(time_fac*t)**2
desired_foot_acc_traj = 6*(1 - time_fac*t) * (cp3 - 2*cp2 + cp1) + 6 * (time_fac*t) * (cp4 - 2*cp3 + cp2)
pt = torch.cat((desired_foot_pos_traj, desired_foot_vel_traj, desired_foot_acc_traj), dim=2)

: 